In [1]:
#imports 
from path import Path
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import sklearn

from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

In [2]:
filepath = Path("../Data/TracfloDataFullSet.csv")
changeorder = pd.read_csv(filepath)
changeorder.head()

,total,paid_total,balance,breakdowns_labor_0_hours,breakdowns_labor_0_rate,breakdowns_material_0_rate,Slug,Total,Labor0Breakdown_quantity0total0,Labor0Breakdown_hourstotal,Labor0Breakdown_ratetotal
0,133000.0,0.0,133000.0,8.0,0.0,0.0,193.0,133000.0,0.0,8.0,165.0
1,27248.0,0.0,27248.0,8.0,0.0,0.0,248.0,27248.0,0.0,8.0,0.0
2,-5000.0,0.0,-5000.0,0.0,0.0,0.0,251.0,0.0,0.0,0.0,0.0
3,13852.0,0.0,13852.0,0.0,0.0,0.0,252.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,8.0,0.0,0.0,253.0,0.0,0.0,8.0,0.0


In [3]:
 #marks 1 for data that is within labor rate of 1 to 800
changeorder["target"]=np.where(changeorder["breakdowns_labor_0_rate"].between(1,800),1,0)

In [4]:
#new df 
changeorder.head()

,total,paid_total,balance,breakdowns_labor_0_hours,breakdowns_labor_0_rate,breakdowns_material_0_rate,Slug,Total,Labor0Breakdown_quantity0total0,Labor0Breakdown_hourstotal,Labor0Breakdown_ratetotal,target
0,133000.0,0.0,133000.0,8.0,0.0,0.0,193.0,133000.0,0.0,8.0,165.0,0
1,27248.0,0.0,27248.0,8.0,0.0,0.0,248.0,27248.0,0.0,8.0,0.0,0
2,-5000.0,0.0,-5000.0,0.0,0.0,0.0,251.0,0.0,0.0,0.0,0.0,0
3,13852.0,0.0,13852.0,0.0,0.0,0.0,252.0,0.0,0.0,0.0,0.0,0
4,0.0,0.0,0.0,8.0,0.0,0.0,253.0,0.0,0.0,8.0,0.0,0


In [5]:
# drop na
changeorder.dropna

<bound method DataFrame.dropna of          total  paid_total   balance  breakdowns_labor_0_hours  \
0     133000.0         0.0  133000.0                       8.0   
1      27248.0         0.0   27248.0                       8.0   
2      -5000.0         0.0   -5000.0                       0.0   
3      13852.0         0.0   13852.0                       0.0   
4          0.0         0.0       0.0                       8.0   
...        ...         ...       ...                       ...   
1747    1330.0         0.0    1330.0                       7.0   
1748   32000.0         0.0   32000.0                       7.0   
1749   17400.0         0.0   17400.0                       7.0   
1750     200.0         0.0     200.0                       0.0   
1751       NaN         0.0       0.0                       0.0   

      breakdowns_labor_0_rate  breakdowns_material_0_rate    Slug     Total  \
0                         0.0                         0.0   193.0  133000.0   
1              

In [6]:
# create x variable 
X = changeorder.copy()
X.drop(["target"], axis=1, inplace=True)
X.head()

,total,paid_total,balance,breakdowns_labor_0_hours,breakdowns_labor_0_rate,breakdowns_material_0_rate,Slug,Total,Labor0Breakdown_quantity0total0,Labor0Breakdown_hourstotal,Labor0Breakdown_ratetotal
0,133000.0,0.0,133000.0,8.0,0.0,0.0,193.0,133000.0,0.0,8.0,165.0
1,27248.0,0.0,27248.0,8.0,0.0,0.0,248.0,27248.0,0.0,8.0,0.0
2,-5000.0,0.0,-5000.0,0.0,0.0,0.0,251.0,0.0,0.0,0.0,0.0
3,13852.0,0.0,13852.0,0.0,0.0,0.0,252.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,8.0,0.0,0.0,253.0,0.0,0.0,8.0,0.0


In [7]:
# create target y variable 
y = changeorder["target"].ravel()

In [8]:
# train test split data 
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [9]:
#scale data 
scaler = StandardScaler()

In [10]:
#fit data 
X_scaler = scaler.fit(X_train)


In [15]:
#scale data 
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [16]:
#random forest 
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model = rf_model.fit(X_train_scaled, y_train)


ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

In [17]:
# Making predictions using the testing data
predictions = rf_model.predict(X_test_scaled)

NotFittedError: This RandomForestClassifier instance is not fitted yet. Call 'fit' with appropriate arguments before using this method.

In [18]:
# Calculating the confusion matrix
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
)

# Calculating the accuracy score
acc_score = accuracy_score(y_test, predictions)

NameError: name 'predictions' is not defined

In [19]:
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, predictions))

Confusion Matrix


NameError: name 'cm_df' is not defined

In [20]:
importances = rf_model.feature_importances_

sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)

NotFittedError: This RandomForestClassifier instance is not fitted yet. Call 'fit' with appropriate arguments before using this method.

In [21]:
from imblearn.over_sampling import SMOTE
X_resampled, y_resampled = SMOTE(random_state=1, ratio=1.0).fit_resample( X_train, y_train)
from collections import Counter
from sklearn.metrics import balanced_accuracy_score
from imblearn.metrics import classification_report_imbalanced

Counter(y_resampled)
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)
confusion_matrix(y_test, y_pred)
print(classification_report_imbalanced(y_test, y_pred))

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
import pydotplus
from IPython.display import Image

In [ ]:
# Create a classifier object
learning_rates = [0.05, 0.1, 0.25, 0.5, 0.75, 1]
for learning_rate in learning_rates:
    classifier = GradientBoostingClassifier(n_estimators=20,
                                            learning_rate=learning_rate,
                                            max_features=5,
                                            max_depth=3,
                                            random_state=0)

    # Fit the model
    classifier.fit(X_train_scaled, y_train.ravel())
    print("Learning rate: ", learning_rate)

    # Score the model
    print("Accuracy score (training): {0:.3f}".format(
        classifier.score(
            X_train_scaled,
            y_train.ravel())))
    print("Accuracy score (validation): {0:.3f}".format(
        classifier.score(
            X_test_scaled,
            y_test.ravel())))
    print()

In [ ]:
# Choose a learning rate and create classifier
classifier = GradientBoostingClassifier(n_estimators=20,
                                        learning_rate=0.25,
                                        max_features=5,
                                        max_depth=3,
                                        random_state=0)

# Fit the model
classifier.fit(X_train_scaled, y_train.ravel())

# Make Prediction
predictions = classifier.predict(X_test_scaled)
pd.DataFrame({"Prediction": predictions, "Actual": y_test.ravel()}).head(20)

In [ ]:
acc_score = accuracy_score(y_test, predictions)
print(f"Accuracy Score : {acc_score}")

In [ ]:

# Generate the confusion matrix
cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"],
    columns=["Predicted 0", "Predicted 1"]
)

# Displaying results
display(cm_df)

In [ ]:
# Generate classification report
print("Classification Report")
print(classification_report(y_test, predictions))

In [ ]:
# Graph tree
dot_data = tree.export_graphviz(
    classifier.estimators_[9, 0],
    out_file=None, filled=True,
    rounded=True,
    special_characters=True,
    proportion=True,
)
graph = pydotplus.graph_from_dot_data(dot_data)
Image(graph.create_png())

In [ ]:
from imblearn.combine import SMOTEENN
smote_enn = SMOTEENN(random_state=1)
X_resampled, y_resampled = smote_enn.fit_resample(X_train, y_train)
Counter(y_resampled)
model = LogisticRegression(random_state=1)
model.fit(X_resampled, y_resampled)
y_pred = model.predict(X_test)


confusion_matrix(y_test, y_pred)

In [ ]:
balanced_accuracy_score(y_test, y_pred)

In [ ]:
from xgboost import XGBClassifier

In [ ]:
# fit model no training data
model = XGBClassifier()
model.fit(X_train, y_train)

In [ ]:
y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]


In [ ]:
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

In [ ]:
importances = model.feature_importances_

importances_data = sorted(zip(model.feature_importances_, X.columns), reverse=True)

In [ ]:
confusion_matrix(y_test, y_pred)

In [ ]:
df_features = pd.DataFrame(importances_data)


In [ ]:
df_top_ten_features = df_features[:10]
df_top_ten_features.columns =['percentage', 'feature name']

In [ ]:
import hvplot.pandas

In [ ]:
df_top_ten_features.hvplot.bar( x='feature name', y='percentage', rot=60)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from keras import losses
import seaborn
from scipy.stats import poisson,skellam

In [ ]:
model = Sequential()
model.add(Dense(32, activation='relu', input_dim=92))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Train the model, iterating on the data in batches of 32 samples
model.fit(X_train_scaled, y_train, epochs=100, batch_size=32)

In [ ]:
model_loss, model_accuracy = model.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

In [ ]:
predicted = model.predict(X_test_scaled)


In [ ]:
results = pd.DataFrame({
    "Actual": y_test,
    "Predicted": predicted.ravel()
})

In [ ]:
results.head(10)


In [ ]:
# Create a DataFrame with the history dictionary
df_epoch = pd.DataFrame(
    model.history, index=range(1, len(model.history["loss"]) + 1)
)
# Plot the loss
df_moon.plot(y="loss")


In [ ]:
number_hidden_nodes = 8
number_input_features = 92

nn = Sequential()
# Hidden layer
nn.add(
    Dense(units=number_hidden_nodes, input_dim=number_input_features, activation="relu")
)
# Output layer
nn.add(Dense(units=1, activation="linear"))

In [ ]:
nn.compile(loss="mean_squared_error", optimizer="adam", metrics=["mse"])

# Train the model
model_1 = nn.fit(X, y, validation_split=0.3, epochs=200)


In [ ]:
plt.plot(model_1.history["loss"][10:200])
plt.title("loss_function - 1 hidden layer")
plt.legend(["loss"])
plt.show()

In [ ]:
number_input_features = 84
hidden_nodes_layer1 = 252
hidden_nodes_layer2 = 126

nn = Sequential()
# First hidden layer
nn.add(
    Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)
# Second hidden layer
nn.add(Dense(units=hidden_nodes_layer2, activation="relu"))
# Output layer
nn.add(Dense(units=1, activation="linear"))


In [ ]:
# Compile model
nn.compile(loss="mean_squared_error", optimizer="adam", metrics=["mse"])

# Fit the model
model_2 = nn.fit(X, y, validation_split=0.3, epochs=20)

In [ ]:
# Train vs test for shallow net
plt.plot(model_1.history["loss"])
plt.plot(model_1.history["val_loss"])
plt.title("loss_function - Training Vs. Validation - 1 hidden layer")
plt.legend(["train", "test"])
plt.show()

In [ ]:
plt.plot(model_2.history["loss"])
plt.plot(model_2.history["val_loss"])
plt.title("loss_function - Training Vs. Validation - 2 hidden layers")
plt.legend(["train", "test"])
plt.show()

In [ ]:
model_loss, model_accuracy = model_2.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")